In [28]:
!pip install numpy
!pip install pandas
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install gensim

     ---------------------------------------- 12.8/12.8 MB 6.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


     ---------------------------------------- 24.0/24.0 MB 2.9 MB/s eta 0:00:00


### Reading the data file

In [8]:
import os
BASE_DIR = os.getcwd()

DATA_DIR= EMBD_PATH = os.path.join(BASE_DIR, "word_count_data")

import pandas as pd
import os
import numpy as np
from copy import deepcopy

unique_region = ['Africa','Asia & Oceania','Europe','Latin America & Caribbean','North America']
LABEL_NAMES = [ 'Refuse','Rethink','Reduce','Reuse','Repair','Refurbish','Remanufacture','Repurpose', 'Recycle','Recover']
year_li = [2014,2015,2016,2017,2018]

In [9]:
org_df = pd.read_csv(os.path.join(DATA_DIR,"combined_output.csv"),low_memory=False)
print(org_df.shape)
print(org_df.region.isnull().sum())
print(org_df.clean_subustype.isnull().sum())

(179452, 36)
0
99797


### Remove stopwords

In [10]:
clean_dic = {"saloon" : 'salon', 
             'supple' : 'supply',
             'hairdresse' : 'hairdresser' , 
             'hairdresserr' : 'hairdresser' , 
             'agricultural': 'agriculture',
             'restuarant':'restaurant',
             'restaurante':'restaurant',
             'resturant' : 'restaurant',
             'restorant' : 'restaurant',
             'storera':'store',
             'good': 'goods',
             'shopkeepe':'shopkeeper',
             'motocicle':'motorcycle',
             'motocycle' : 'motorcycle',
             'moto':'motor',  
             'motorr' : 'motor',
             'cybercaf':'cybercafe',
             'cloth' : 'clothes',
             'onlie':'online',
             'soft drink': 'softdrink',
             'second hand' : 'secondhand',
             'fast food':'fastfood',
             'show room': 'showroom',
             'tomatoe':'tomato',
             'breed' :'breeding',
             'hand bag': 'handbag',
             'cyber cafe': 'cybercafe',
             'caf' : 'cafe',
             'internet caf':'cybercafe',
             'cafee' : 'cafe',
             'dry cleaning' :'laundry',
             'second class':'secondhand',
             'ready wear' : 'readywear',
             'clean' : 'cleaning',
             'sew': 'sewing',
             'vend': 'vendor',
             'architecture' : 'architectural',
             'consult' : 'consultancy',
             'clotheing' : 'clothing',
             'clothesing' : 'clothing',
             'clothee' : 'clothes',
             'vendoror' : 'vendor',
             'cell phone' : 'mobile phone',
             'cel phone' : 'mobile phone',
             'consultancyancy' : 'consultancy',
             'consultancying' : 'consultant',
             'grocery grocery': 'grocery',
             'health care' : 'healthcare',
             'hair stylish' : 'hairstylist',
             'sewinging' : 'sewing',
             'sewinger': 'sewing',
             'day care' :'daycare',
             'breedinging' : 'breeding',
             'clothese' : 'clothes'}

#https://stackoverflow.com/questions/51240862/find-and-replace-multiple-words-in-a-file-python
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [14]:
#https://towardsdatascience.com/text-normalization-with-spacy-and-nltk-1302ff430119

#spaCy Code Initialization:
import spacy

nlp = spacy.load('en_core_web_sm')

custom_words_to_remove= [' need', 'level', 'barranquilla', 'know', 
                         'basic','sell', 'sale', 'buy',"shop",  "business" , 
                          "customer", 'person', 'single', 'service',"consumer","store","typical",
                         "company", "general","private",'public',"resident","people","product","base"
                        "living", "area","want","like","room",'ld','ltd','american', 'item', 'answer', 'mean','michel',
                         'jose','de', 'etc', 'da','use','outer','inner','stall','don',
                        'non','predominate','middle','domain','detailed','activity','rican','base']
                        
def spacy_process(text1):
    if(text1==''):
        return ''
    else:
        text = str(text1)
        doc = nlp(text)

        #Tokenization and lemmatization are done with the spacy nlp pipeline commands
        lemma_list = []
        for token in doc:
            lemma_list.append(token.lemma_)
        #print("Tokenize+Lemmatize:")

        #Filter the stopword
        filtered_sentence =[] 
        for index, word in enumerate(lemma_list):
            lexeme = nlp.vocab[word]
            if lexeme.is_stop == False and len(word)>1 and  word not in custom_words_to_remove:
                filtered_sentence.append(word) 

        combined_word = " ".join(filtered_sentence)
        return replace_all(combined_word, clean_dic)

text = " we will show how to ducks good supple buy  sell laundry fishing stopwords using spacy i ltd tmp library know house basic time level housing soft drink weld"
print("Original text  : ",text)
print("Text after removing stopwords  : ",spacy_process(text))

text = ""
print("Original text  : ",text)
print("Text after removing stopwords  : ",spacy_process(text))

Original text  :   we will show how to ducks good supple buy  sell laundry fishing stopwords using spacy i ltd tmp library know house basic time level housing soft drink weld
Text after removing stopwords  :  duck goods supply laundry fishing stopword spacy tmp library house time housing softdrink weld
Original text  :  
Text after removing stopwords  :  


### Data Cleaning /Checking for duplicates duplicates

In [16]:
org_df['clean_subustype_wo_stopwords'] = org_df['clean_subustype'].apply(spacy_process)
org_df['clean_subustype_wo_stopwords'].head(5)

org_df['clean_subustype_wo_stopwords'].replace('', np.nan, inplace=True)

if 'Unnamed: 0' in org_df.columns:
    print(org_df.columns)
    org_df= org_df.drop(['Unnamed: 0'], axis=1, errors='ignore')

org_df.to_csv(os.path.join(DATA_DIR,"combined_df_wo_stowords.csv"),index=False)

is_unique = not org_df.duplicated(['setid','yrsurv']).any()
print(is_unique)

True


### Extracting valid data for wordcount

In [17]:
q3_val = 0.9
op_df= pd.read_csv(os.path.join(DATA_DIR,"combined_df_wo_stowords.csv"),low_memory=False)

if(q3_val == 0.75):
    Q3_fileName = 'table_Q3_75.csv'
    Valid_df_fileName = "Valid_df_75_wo_stowords.csv"

    topic_modelling_fileName= "Topic_modelling_output_75.csv"
    topic_modelling_formatted_fileName="Topic_modelling_output_formatted_75.csv"

    word_count_fileName= "word_count_output_75.csv"
    word_count_formatted_fileName="word_count_result_75.csv"

else:
    Q3_fileName = 'table_Q3_90.csv'
    Valid_df_fileName = "Valid_df_90_wo_stowords_wo_Rs.csv"

    topic_modelling_fileName= "Topic_modelling_output_90_wo_Rs.csv"
    topic_modelling_formatted_fileName="Topic_modelling_output_formatted_90_wo_Rs.csv"

    word_count_fileName= "word_count_output_90_wo_Rs.csv"
    word_count_formatted_fileName="word_count_result_90_wo_Rs.csv"

In [18]:
valid_unique_id = []
valid_weighted_sum = []

def extract_valid(raw_df, df_q3_vals):
    # Remove rows with missing values in the 'clean_subustype_wo_stopwords' column
    raw_df.dropna(subset=['clean_subustype_wo_stopwords'], inplace=True)
    
    # Create a new DataFrame with the same columns as raw_df
    new_df = pd.DataFrame(columns=raw_df.columns)
    
    insertion_index = 0

    # Iterate over unique regions
    for i in range(len(unique_region)):
        # Iterate over year_li values
        for j in range(len(year_li)):
            # Filter df_q3_vals based on region and year
            valid_q3 = df_q3_vals[(df_q3_vals['region'] == unique_region[i]) & (df_q3_vals['yrsurv'] == year_li[j])]
            
            # Filter raw_df based on region and year
            raw_reg = raw_df[(raw_df['region'] == unique_region[i]) & (raw_df['yrsurv'] == year_li[j])]

            # Iterate over rows in raw_reg
            for index, row in raw_reg.iterrows():
                # Check if weighted_sum is greater than or equal to the threshold in valid_q3
                if row['weighted_sum'] >= valid_q3['weighted_sum'].values[0]:
                    # Add the row to the new DataFrame
                    new_df.loc[insertion_index] = row
                    insertion_index += 1
                    
                    # Store the valid unique ID and weighted sum in the respective lists
                    valid_unique_id.append(row['setid'])
                    valid_weighted_sum.append(row['weighted_sum'])
                    
    return new_df

# Read df_q3 from a CSV file
df_q3 = pd.read_csv(os.path.join(DATA_DIR, Q3_fileName))

# Call the extract_valid function to get the valid DataFrame
valid_df = extract_valid(op_df, df_q3)  

# Save the valid DataFrame to a CSV file
valid_df.to_csv(os.path.join(DATA_DIR,Valid_df_fileName),index=False)  

##90 percentile

In [19]:
print(op_df.shape, valid_df.shape)
print(op_df.shape[0]-valid_df.shape[0])

valid_df = pd.read_csv(os.path.join(DATA_DIR,Valid_df_fileName))
df = valid_df.copy()
df.shape

(78078, 37) (7670, 37)
70408


(7670, 37)

### Storing the data with column to specify if counted for topic modelling

In [20]:
op_df['is_valid_for_Topic_modelling'] = 0 

index_list=[]
for i in range(len(valid_unique_id)):
    index = op_df.loc[(op_df['setid'] == valid_unique_id[i]) & (op_df['weighted_sum'] == valid_weighted_sum[i])].index[0]
    index_list.append(index)
    
op_df.loc[index_list,'is_valid_for_Topic_modelling'] = 1
op_df.to_csv(os.path.join(DATA_DIR,"combined_df_wo_stowords_topic_modelling_valid_indicator.csv"),index=False)

print(op_df.shape)
op_df.columns

(78078, 38)


Index(['setid', 'ID', 'ctryalp', 'region', 'country', 'yrsurv',
       'sestart_encoded', 'sestart', 'language', 'age', 'gender', 'GEMEDUC',
       'GEMHHINC', 'GEMWORK', 'opport', 'fearfail', 'suskill', 'nbgoodc',
       'nbstatus', 'nbmedia', 'knowent', 'bstart', 'subustype',
       'eng_subustype', 'clean_subustype', 'Refuse', 'Rethink', 'Reduce',
       'Reuse', 'Repair', 'Refurbish', 'Remanufacture', 'Repurpose', 'Recycle',
       'Recover', 'weighted_sum', 'clean_subustype_wo_stopwords',
       'is_valid_for_Topic_modelling'],
      dtype='object')

### updating the topic modelling indiacator for the raw survery data

In [21]:
raw_input_df= pd.read_csv(os.path.join(BASE_DIR,"raw_survey_data_selected_cols.csv"),low_memory=False)
raw_input_df.shape

is_raw_unique = not raw_input_df.duplicated(['setid','yrsurv']).any()
print(is_raw_unique)

True


In [23]:
#Encoding sestart column
def sestart_categorizer(val):
    if val == 'Yes, currently trying to start':
        return 1
    elif val == 'Yes, currently leading':
        return 1
    elif val == 'Yes, trying to start AND leading':
        return 1
    elif val == 'No':
        return 0
    else:
        return -2
    
raw_input_df['sestart_encoded'] =  raw_input_df['sestart'].apply(sestart_categorizer)

In [24]:
## we only need selected columns
selected_op_df = op_df[['setid','yrsurv', 'language','clean_subustype',
       'Refuse', 'Rethink', 'Reduce', 'Reuse', 'Repair', 'Refurbish',
       'Remanufacture', 'Repurpose', 'Recycle', 'Recover', 'weighted_sum',
       'clean_subustype_wo_stopwords', 'is_valid_for_Topic_modelling']]

merged_df = pd.merge(raw_input_df, selected_op_df, on=['setid','yrsurv'], how='left')
merged_df[['weighted_sum','is_valid_for_Topic_modelling']] = merged_df[['weighted_sum','is_valid_for_Topic_modelling']].fillna(0)

## To write in a specific format
merged_df_formatted  = merged_df[['setid', 'ID', 'ctryalp', 'region', 'country', 'yrsurv', 'sestart_encoded','sestart', 
                                  'language','age','gender', 'GEMEDUC', 'GEMHHINC', 'GEMWORK', 'opport', 'fearfail',
       'suskill', 'nbgoodc', 'nbstatus', 'nbmedia', 'knowent', 'bstart',
       'subustype', 'eng_subustype', 
       'clean_subustype', 'Refuse', 'Rethink', 'Reduce', 'Reuse', 'Repair',
       'Refurbish', 'Remanufacture', 'Repurpose', 'Recycle', 'Recover',
       'weighted_sum', 'clean_subustype_wo_stopwords',
       'is_valid_for_Topic_modelling']]

#merged_df_formatted.to_csv(os.path.join(DATA_DIR,"raw_df_topic_modelling_valid_indicator.csv"),index=False)

merged_df_formatted.head(4)

,setid,ID,ctryalp,region,country,yrsurv,sestart_encoded,sestart,language,age,...,Reuse,Repair,Refurbish,Remanufacture,Repurpose,Recycle,Recover,weighted_sum,clean_subustype_wo_stopwords,is_valid_for_Topic_modelling
0,1.141000e+09,1000115.0,US,North America,United States,2014.0,-2,NaN,NaN,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
1,1.141000e+09,1000121.0,US,North America,United States,2014.0,-2,NaN,NaN,54.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
2,1.141000e+09,1000319.0,US,North America,United States,2014.0,-2,NaN,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
3,1.141000e+09,1000460.0,US,North America,United States,2014.0,-2,NaN,NaN,56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0


In [25]:
merged_df_formatted.columns

Index(['setid', 'ID', 'ctryalp', 'region', 'country', 'yrsurv',
       'sestart_encoded', 'sestart', 'language', 'age', 'gender', 'GEMEDUC',
       'GEMHHINC', 'GEMWORK', 'opport', 'fearfail', 'suskill', 'nbgoodc',
       'nbstatus', 'nbmedia', 'knowent', 'bstart', 'subustype',
       'eng_subustype', 'clean_subustype', 'Refuse', 'Rethink', 'Reduce',
       'Reuse', 'Repair', 'Refurbish', 'Remanufacture', 'Repurpose', 'Recycle',
       'Recover', 'weighted_sum', 'clean_subustype_wo_stopwords',
       'is_valid_for_Topic_modelling'],
      dtype='object')

## WordCount

In [40]:
df = pd.read_csv(os.path.join(DATA_DIR,Valid_df_fileName))
li=[]

word_count_df = pd.DataFrame()

for region in unique_region:

    count_df = pd.DataFrame()
    selected_df = (df[df['region'] == region])
    word_list = selected_df['clean_subustype_wo_stopwords'].values
    print(region)
    word_count_op = count_words(word_list)
    print()

    w_li = []
    p_li = []
    for word in word_count_op:
        w_li.append(word[0])
        p_li.append(word[1])

    count_df['word'] = w_li
    count_df['share'] = p_li
    count_df['region'] = region

    li.append(region)
    li.append(word_count_op)
    
    word_count_df =  pd.concat([word_count_df, count_df], axis=1)
word_count_df.to_csv(os.path.join(DATA_DIR, word_count_fileName), index=False)

word_count_df = pd.DataFrame()
word_count_df['result'] = li
word_count_df.to_csv(os.path.join(DATA_DIR,word_count_formatted_fileName), index=False)

Africa
Percentage share of each word : [('trade', 3.66), ('repair', 2.15), ('local', 1.32), ('car', 1.27), ('clothes', 1.18), ('secondhand', 1.07), ('spare', 0.96), ('small', 0.94), ('food', 0.88), ('clothing', 0.88), ('open', 0.85), ('tertiary', 0.83), ('retail', 0.8), ('vegetable', 0.77), ('bar', 0.72)]

Asia & Oceania
Percentage share of each word : [('repair', 2.21), ('car', 1.67), ('online', 1.56), ('woman', 1.5), ('clothes', 1.43), ('accessory', 0.92), ('household', 0.91), ('house', 0.86), ('food', 0.83), ('phone', 0.81), ('home', 0.74), ('clothing', 0.68), ('mobile', 0.66), ('local', 0.65), ('provide', 0.64)]

Europe
Percentage share of each word : [('repair', 2.67), ('car', 1.92), ('online', 1.69), ('trade', 1.51), ('individual', 1.26), ('clothing', 1.09), ('client', 1.08), ('industry', 0.91), ('furniture', 0.9), ('food', 0.77), ('bar', 0.75), ('manufacture', 0.74), ('work', 0.72), ('cleaning', 0.7), ('production', 0.7)]

Latin America & Caribbean
Percentage share of each word 